In [ ]:
from models import *
import pandas as pd
import glob

In [ ]:
!rm database.db

In [ ]:
print db
db.create_all()

Constituency : models.Constituency

Results : models.Results

Party : model.Party

Candidates : models.Candidates

In [ ]:
db.session.rollback()

In [ ]:
# JSON Files for Boundaries.
files = glob.glob('./election_json/boundaries/*.json')
files.sort()

In [ ]:
# XLS File with MP names.
data_1 = './current_mp.xls'
# Fetch names, sort and reindex.
names = pd.read_excel(data_1)
names = names.sort(columns=u'Constituency') # Sorted into constutuency order
names.index = range(0,len(names))

In [ ]:
# XLS File with Constituency voting 2010.
"""
So this short piece of code reads the excel file, then strips out all the rows that have a NaN in the first column.
This leaves us, for each constituency, the name, the total amount registered and finally the last General election winner
(i.e. three rows each). Then a nice clean reindex for sanity.

Then the plan is simiple. To create a series of N by 1 data frames for easy indexing for the database.
"""
# Prelim work
data_2 = './constituency.xls'
cons = pd.read_excel(data_2)
cons = cons[pd.notnull(cons['General election results 2010'])]
cons.index = range(0,len(cons))
idx_arr = range(1,len(files)*3,3)
# Turnout Percantage
turnout = cons['Turnout'][pd.notnull(cons['Turnout'])]
turnout.index = range(0,len(turnout))
# Number of votes for the winning candidate
votes = cons['Votes'][idx_arr]
votes.index = range(0,len(votes))
# Perctange Share for the winning candidate
share = cons['Share'][idx_arr]
share.index = range(0,len(share))
# Total number of registered votes for each con
total = cons['General election results 2010'][idx_arr]
total.index = range(0,len(total))
# Percentage share
share = cons['Share'][idx_arr]

In [ ]:
for idx, afile  in enumerate(files):
    # Basic Constituency data
    con = Constituency(name=afile.rsplit('/')[-1].rsplit('.')[0])
    con.numvotes = total[idx] # Total number of registered votes at last general election
    # Results
    con.Results = Results
    con.Results.turnout = turnout[idx] # Percentage Turnout
    con.Results.year = '2010-5-6'
    # Candidates
    con.Candidates = Candidates
    con.Candidates.MPname = names['First name'][idx] + ' ' + names['Last name'][idx]
    con.Candidates.numvotes = votes[idx]
    con.Candidates.MPparty = Party
    con.Candidates.MPparty.name = names['Party'][idx]
    con.Candidates.share = share[idx]
    db.session.add(con)